# Tick Strategy
_Version 0.1_

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# load our helper functios
import simtools as st
import tick as tk
from getstock import *
from simtools import *

matplotlib.rcParams[ 'figure.figsize' ] = ( 16, 9 )

### Load TAQ Data

We'll assume we have already done some cleaning by using our Tick Data Setup or similar code

In [2]:
# set up some variables
# NOTE: tickfile is a merged TAQ file!!!
tickfile = 'APPL-TAQ-Sep03.zip'
#test_day = '09/03/19'
taq = pd.read_pickle(tickfile)

In [3]:
taq

# filter on date
#day = taq[test_day]

,exch,bid_px,bid_size,ask_px,ask_size,qu_cond,qu_seqnum,natbbo_ind,qu_cancel,qu_source,symbol,suffix_x,suffix_y,trade_size,trade_px
2019-09-03 04:00:00.071606324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAPL,NaN,NaN,48.0,206.20
2019-09-03 04:00:00.072720314,P,165.00,2.0,0.00,0.0,R,2162.0,2.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
2019-09-03 04:00:00.072731184,P,165.00,2.0,221.00,1.0,R,2163.0,4.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
2019-09-03 04:00:00.072741263,P,171.00,1.0,221.00,1.0,R,2164.0,4.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
2019-09-03 04:00:00.072752330,P,181.56,1.0,221.00,1.0,R,2165.0,4.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-03 19:59:41.893688808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAPL,NaN,NaN,151.0,205.98
2019-09-03 19:59:41.893689963,K,205.98,1.0,206.12,5.0,R,35238641.0,4.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
2019-09-03 19:59:58.764819745,K,205.89,9.0,206.12,5.0,R,35238699.0,4.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
2019-09-03 19:59:58.764822342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAPL,NaN,NaN,100.0,205.98


## Run the Simulator

In [ ]:
# without the tick signal...
results = tk.algo_loop( taq )

2019-10-27 17:42:07:701854: Beginning Tick Strategy run
Average stock spread for sample: 1.9864
2019-10-27 17:42:08:872707: starting main loop


In [ ]:
results['fair_values'].plot()

results['midpoints'].plot()
day.trade_px.plot()

In [ ]:
data = results['trades']

data[pd.notna(data.price)]['shares'].sum()

results.keys()

results['day_vwap']

results['avg_price']

#slippage vs. vwap
results['day_vwap'] - results['avg_price']

results['trades'].groupby(['trade_type']).count()